In [ ]:
# Neural Networks From Scratch (in Python)


In [7]:
# Import Libraries
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load and Preprocess Data
iris = datasets.load_iris()
X = iris.data  # Features
y = iris.target.reshape(-1, 1)  # Labels reshaped as column vector

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Neural Network Functions

# Initialize parameters for the neural network
def initialize_parameters(input_dim, hidden_dim, output_dim):
    np.random.seed(42)
    W1 = np.random.randn(hidden_dim, input_dim) * 0.01
    b1 = np.zeros((hidden_dim, 1))
    W2 = np.random.randn(output_dim, hidden_dim) * 0.01
    b2 = np.zeros((output_dim, 1))
    return W1, b1, W2, b2

# Activation Functions
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # For numerical stability
    return expZ / np.sum(expZ, axis=0, keepdims=True)

# Forward Propagation
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X.T) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    return A1, A2

# Cross-Entropy Loss Function (Fixed)
def compute_loss(A2, Y):
    m = Y.shape[0]  # Number of examples
    log_probs = np.log(A2 + 1e-9)  # Avoid log(0)
    loss = -np.sum(Y.T * log_probs) / m
    return loss

# Backpropagation (Fixed)
def backpropagation(X, Y, A1, A2, W1, W2):
    m = X.shape[0]  # Number of examples
    
    dZ2 = A2 - Y.T
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    
    dZ1 = np.dot(W2.T, dZ2) * A1 * (1 - A1)
    dW1 = np.dot(dZ1, X) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m
    
    return dW1, db1, dW2, db2

# Train the neural network using gradient descent
def gradient_descent(X, Y, W1, b1, W2, b2, learning_rate=0.1, num_epochs=1000):
    for epoch in range(num_epochs):
        A1, A2 = forward_propagation(X, W1, b1, W2, b2)
        loss = compute_loss(A2, Y)
        
        dW1, db1, dW2, db2 = backpropagation(X, Y, A1, A2, W1, W2)
        
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss = {loss:.4f}")
            
    return W1, b1, W2, b2

# Model Training
input_dim = 4   # Features
hidden_dim = 10 # Neurons in hidden layer
output_dim = 3  # Classes

W1, b1, W2, b2 = initialize_parameters(input_dim, hidden_dim, output_dim)

W1, b1, W2, b2 = gradient_descent(X_train, y_train, W1, b1, W2, b2)

# Make Predictions
def predict(X, W1, b1, W2, b2):
    _, A2 = forward_propagation(X, W1, b1, W2, b2)
    predictions = np.argmax(A2, axis=0)
    return predictions

# Predict on test data and evaluate the model
y_pred = predict(X_test, W1, b1, W2, b2)
y_true = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred == y_true) * 100
print(f"Test Set Accuracy: {accuracy:.2f}%")


Epoch 0: Loss = 1.0987
Epoch 100: Loss = 1.0831
Epoch 200: Loss = 0.7748
Epoch 300: Loss = 0.5100
Epoch 400: Loss = 0.4189
Epoch 500: Loss = 0.3459
Epoch 600: Loss = 0.2844
Epoch 700: Loss = 0.2367
Epoch 800: Loss = 0.2012
Epoch 900: Loss = 0.1751
Test Set Accuracy: 100.00%
